## data analysis and transformation

features:
- date_time - время показа рекламы
- oaid_hash - хэш юзера
- zone_id - id зоны, где зона - место на сайте для размещения рекламы
- banner_id - id баннера, где баннер - сама реклама
- os_id - id операционной системы
- country_id - id страны
- clicks - был ли клик

to calculate probs:
- banner_id0 - нулевой баннер в “стакане” баннеров
- banner_id1 - перый баннер в “стакане” баннеров
- g0 - стандартное отклонение предикта с banner_id0
- g1 - стандартное отклонение предикта с banner_id1
- coeff_sum0 - сумма коэффициентов для banner_id0
- coeff_sum1 - сумма коэффициентов для banner_id1

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Знаем, что 'impressions' не нужны.
# по условию ревард не за зависит от цены клика, так что уберем 'rate0' и 'rate1'
data_raw = pd.read_csv('data.csv', parse_dates=['date_time']).drop(['campaign_clicks', 'impressions', 'rate0', 'rate1'], axis=1)
data_raw.head()

,date_time,zone_id,banner_id,oaid_hash,os_id,country_id,banner_id0,g0,coeff_sum0,banner_id1,g1,coeff_sum1,clicks
0,2021-09-27 00:01:30,0,0,5664530014561852622,0,0,1240,0.035016,-7.268846,0,0.049516,-5.369901,1
1,2021-09-26 22:54:49,1,1,5186611064559013950,0,1,1,0.054298,-2.657477,269,0.031942,-4.449220,1
2,2021-09-26 23:57:20,2,2,2215519569292448030,0,0,2,0.014096,-3.824875,21,0.014906,-3.939309,1
3,2021-09-27 00:04:30,3,3,6262169206735077204,1,1,3,0.015232,-3.461357,99,0.050671,-3.418403,1
4,2021-09-27 00:06:21,4,4,4778985830203613115,1,0,4,0.051265,-4.009026,11464230,0.032005,-2.828797,1


In [3]:
def clear_ds(df):
    print('Check nans:')
    print(df.isna().sum(), end='\n\n')

    # delete rows with nans
    data_without_nans = df.drop(df[df.isna().any(axis=1)].index)

    # знаем, что есть выброс, уберем его тоже
    data_without_nans_outliers = data_without_nans.drop(data_without_nans[data_without_nans['date_time'].dt.date
                                                        == pd.to_datetime('2021-09-01').date()].index)

    # уберем строчки где 'banner_id0' != 'banner_id'
    data_cleared =  data_without_nans_outliers.drop(data_without_nans_outliers[data_without_nans_outliers['banner_id0']
                                                    != data_without_nans_outliers['banner_id']].index)
    data_cleared.reset_index(drop=True, inplace=True)

    return data_cleared

In [4]:
data_cleared = clear_ds(data_raw)

Check nans:
date_time         0
zone_id           0
banner_id         0
oaid_hash         0
os_id             0
country_id        0
banner_id0        0
g0               69
coeff_sum0       69
banner_id1        0
g1            19744
coeff_sum1    19744
clicks            0
dtype: int64



In [5]:
def analysis(df):
    # посмотрим на новые фичи
    new_features = ['g0', 'g1', 'coeff_sum0', 'coeff_sum1']
    for feature in new_features:
        print(f"{feature}: min_val: {min(df[feature].values)}, max_val: {max(df[feature].values)}")

In [6]:
analysis(data_cleared)

g0: min_val: -0.0176373981227117, max_val: 691.088787242959
g1: min_val: -0.0698389748079466, max_val: 691.088452897728
coeff_sum0: min_val: -8.31140713055213, max_val: 0.0
coeff_sum1: min_val: -8.42782707927554, max_val: 0.475618061292297


- Удалим отрицательные g0 и g1, так как они будут выступать в роли стандартного отклонения.
- Разделим на train и test. 
- От дат оставим только часы, как делали в предыдущих работах
- Применим one-hot (фичи все категориальные)

In [7]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack

In [8]:
def transform(df):
    # remove invalid g0 and g1
    df = df.drop(df[(df['g0'] < 0) & (df['g1'] < 0)].index)
    df.reset_index(drop=True, inplace=True)

    #train test
    x_train = df[df['date_time'].dt.date != pd.to_datetime('2021-10-02').date()]
    y_train = x_train['clicks']
    x_train = x_train.drop('clicks', axis=1)

    x_test = df[df['date_time'].dt.date == pd.to_datetime('2021-10-02').date()]
    y_test = x_test['clicks']
    x_test = x_test.drop('clicks', axis=1)

    #remove dates, add hours
    x_train['hour'] = x_train['date_time'].dt.hour
    x_train = x_train.drop('date_time', axis=1)

    x_test['hour'] = x_test['date_time'].dt.hour
    x_test = x_test.drop('date_time', axis=1)

    # уберем фичи, которые не будут использоваться для предсказания 
    x_train = x_train.drop(['coeff_sum0', 'coeff_sum1', 'g0', 'g1', 'banner_id0', 'banner_id1'],axis=1) #banner_id
    x_test_pi0 = x_test.drop(['coeff_sum0', 'coeff_sum1', 'g0', 'g1', 'banner_id0', 'banner_id1'],axis=1) #banner_id
    
    x_test_pi1 = x_test.drop(['coeff_sum0', 'coeff_sum1', 'g0', 'g1', 'banner_id0', 'banner_id'],axis=1) #banner_id1
    x_test_pi1['banner_id'] = x_test_pi1['banner_id1']
    x_test_pi1 = x_test_pi1.drop('banner_id1', axis=1)

    # понадобится, чтобы считать вероятности для p0 и p1
    distrib_parameters = x_test[['coeff_sum0', 'coeff_sum1', 'g0', 'g1']]

    #interactions
    for context_feature in ['os_id', 'country_id', 'oaid_hash']:
        for adv_feature in ['hour', 'zone_id', 'banner_id']:
            inter_feature = context_feature + '_' + adv_feature
            x_train[inter_feature] = x_train[context_feature].astype(str)+ '_' + x_train[adv_feature].astype(str)
            x_test_pi0[inter_feature] = x_test_pi0[context_feature].astype(str)+ '_' + x_test_pi0[adv_feature].astype(str)
            x_test_pi1[inter_feature] = x_test_pi1[context_feature].astype(str)+ '_' + x_test_pi1[adv_feature].astype(str)

    
    #one_hot
    train_sparse = []
    test_sparse_pi0 = []
    test_sparse_pi1 = []
    
    for feature in list(x_train.columns):
        encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore').fit(x_train[[feature]])
        train_encoded = encoder.transform(x_train[[feature]])
        train_sparse.append(train_encoded)
        
        test_pi0_encoded = encoder.transform(x_test_pi0[[feature]])
        test_sparse_pi0.append(test_pi0_encoded)

        test_pi1_encoded = encoder.transform(x_test_pi1[[feature]])
        test_sparse_pi1.append(test_pi1_encoded)
    
    return hstack(train_sparse), y_train, hstack(test_sparse_pi0), hstack(test_sparse_pi1), y_test, distrib_parameters

train_sparse, y_train, test_sparse_pi0, test_sparse_pi1, y_test, distrib_parameters = transform(data_cleared)

## получим оценку 

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, roc_auc_score

In [10]:
# как в первом дз
model = LogisticRegression(penalty='l2',
                           C=1.0,
                           solver='liblinear')

In [11]:
model.fit(train_sparse, y_train)

LogisticRegression(solver='liblinear')

Проверим качество эстиматора:

In [12]:
preds = model.predict_proba(test_sparse_pi0)[:, 1]

In [13]:
log_loss(y_test, preds), roc_auc_score(y_test, preds)

(0.12536785397565942, 0.8262897351234961)

Качество даже лучше, чем в первом дз.

## CIPS

Посчитаем новые средние:

In [14]:
from scipy.special import logit
from scipy.stats import norm

In [15]:
probs0 = model.predict_proba(test_sparse_pi0)[:, 1]
new_mean0 = logit(probs0)

probs1 = model.predict_proba(test_sparse_pi1)[:, 1]
new_mean1 = logit(probs1)

Итак, нужно оценить вероятность того, что одна случайная величина больше другой.  
$$
x_1 \sim N(a_1, \sigma_1), x_2 \sim N(a_2, \sigma_2)
$$
$$
P(x_1 > x_2) = 1 - P(x_1 - x_2 < 0) = 1 - F_{x_1-x_2}(0) 
$$
$$
x_1 - x_2 \sim N(a1-a2, \sigma_1 + \sigma_2)
$$

In [16]:
def calc_prob(a1, s1, a2, s2):
    return 1 - norm.cdf(0, loc=a1-a2, scale=np.sqrt(s1**2 + s2**2) + 1e-3)

In [17]:
pi0 = calc_prob(distrib_parameters['coeff_sum0'], 
                distrib_parameters['g0'],
                distrib_parameters['coeff_sum1'],
                distrib_parameters['g1'])

pi1 = calc_prob(new_mean0,
                distrib_parameters['g0'], 
                new_mean1,
                distrib_parameters['g1'])

In [18]:
# reward у нас 0 или 1 в зависимости от клика
lambda_ = 10
cips = np.mean(y_test * np.clip(pi1/(pi0+1e-3), a_min=None, a_max=lambda_))
print(f"cips: {cips}")

cips: 0.06212541678077257
